In [6]:
'''
Based on: https://github.com/fmfn/BayesianOptimization/blob/master/examples/xgboost_example.py
Computes the best parameters for XGB model optimization
'''

# imports
import pandas as pd
import numpy as np
import os
import gc
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit

import os
from slacker import Slacker
# slack secrets (in your ~/.bashrc)
webhook_url = os.environ.get('SLACK_URL')
slacker = Slacker(os.environ.get('SLACK_TOKEN'))

In [7]:
def XGBcv(max_depth, gamma, min_child_weight, max_delta_step, subsample, colsample_bytree, alpha):
    folds = 5
    paramt = {
        'eta': 0.1,
        'verbose_eval': None,
        'silent': 0,
        'objective': 'reg:linear',
        'booster': 'gbtree',
        'eval_metric': 'mae',
        'updater': 'grow_gpu',
#         'eta': max(eta, 0),
        'max_depth': int(max_depth),
        'alpha': max(alpha, 0),
        'gamma': max(gamma, 0),
        'subsample': max(min(subsample, 1), 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'min_child_weight': int(min_child_weight),
        'max_delta_step': int(max_delta_step)
    }
    
    out = xgb.cv(paramt,
           dtrain,
           num_boost_round=300,
           folds=tscv.split(dtrain),
           callbacks=[xgb.callback.early_stop(10)])
    
    output = -out['test-mae-mean'].values[-1]
    del out
    gc.collect()
    
    return output


In [8]:
# from past experiments
{'target': [-207885.453125, -208155.27083333334, -208065.26041666666, -8482.436034999999, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -187276.90625, -7297.8352863333339, -187186.921875, -13187.901367333334, -187186.921875, -187186.921875, -187186.921875, -187186.921875, -187186.921875, -187186.921875], 'alpha': [9.609593275137133, 0.6613706829779054, 9.99437912773472, 0.2082503541460734, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.926950421589801, 8.8964024320545558, 9.9943791277347191, 0.58608203018219007, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191], 'colsample_bytree': [0.3695897782674733, 0.8602917861499724, 0.26467482205195547, 0.2632600769188274, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.93220963838148418, 0.31851666997270223, 0.26467482205195547, 0.17611852499412145, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547], 'gamma': [9.865587065065188, 9.74686134560233, 6.417811142881344, 9.986369268602083, 6.417811142881344, 6.417811142881344, 6.417811142881344, 6.417811142881344, 6.417811142881344, 6.417811142881344, 0.2500394735324063, 0.52317867353906866, 6.4178111428813436, 0.6138584518188217, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436], 'max_delta_step': [4.319369982200314, 1.4106219539239218, 2.8567339088646717, 0.62666064228038, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 1.4850239461478287, 0.46898495419691544, 2.8567339088646717, 0.19645230111910383, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717], 'max_depth': [5.051048152090837, 14.840043411931143, 9.996819662914094, 5.649088797554254, 9.996819662914094, 9.996819662914094, 9.996819662914094, 9.996819662914094, 9.996819662914094, 9.996819662914094, 6.2644943972420322, 14.315316864637497, 9.9968196629140937, 14.195801982264026, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937], 'min_child_weight': [19.90604381074864, 1.5376515645602162, 6.3381577144228505, 19.934027009976592, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 1.1124257567329519, 19.960599578245827, 6.3381577144228505, 19.570587168476809, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505], 'subsample': [0.8722713229598846, 0.5416489553632888, 0.7964743895201556, 0.6059873611138029, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.89264497955995503, 0.63825065926173097, 0.79647438952015559, 0.96166146546166686, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559]}


In [9]:
params = {       'max_depth': (5, 15),
                 'gamma': (0.0, 10.0),
                 'min_child_weight': (1, 20),
                 'max_delta_step': (0, 5),
                 'subsample': (0.5, 1.0),
                 'colsample_bytree' :(0.1, 1.0),
                 'alpha': (0, 10)
               }

XGB_BOpt = BayesianOptimization(XGBcv, params)
#XGB_BOpt.initialize(init_values)

df = pd.read_csv('CSV_backups/ALL-sales.csv',nrows=100000)

cols = df.columns
ind2remove = ['Unnamed: 0', 'address', 'area_name', 'date_listed', 'id', 'listed_diff_id', 'lookup_address',
              'origin_url', 'neighborhood', 'zipcode', 'luxurious', 'transaction_status', 'transaction_type',
              'zestimate_sale']
factors = np.setdiff1d(cols, ind2remove)

dtrain = xgb.DMatrix(df[factors].values, label=df.price, feature_names=factors)
tscv = TimeSeriesSplit(n_splits=5)

# per link below i need to use Upper Confidence Bound and add some alpha (square of stdev), otherwise it starts to loop
# https://github.com/fmfn/BayesianOptimization/issues/10 
gp_params = { 'alpha' : 400000}
XGB_BOpt.maximize(init_points=5, n_iter=10, acq='ucb', kappa=50, **gp_params)

/home/ilya/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Initialization
----------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 


/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    1 | 04m35s | -1674.97685 |    2.6456 |             0.6759 |    6.4589 |           0.4356 |     10.4881 |            16.0428 |      0.9893 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    2 | 04m47s | -9962.74479 |    7.7423 |             0.2290 |    4.3759 |           0.1011 |     12.1519 |            11.0490 |      0.8996 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    3 | 05m31s | -187006.93229 |    4.5615 |             0.9502 |    8.9177 |           4.1631 |     11.0276 |            11.7928 |      0.7307 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.

/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    6 | 03m01s | -187276.90625 |    9.9270 |             0.9322 |    0.2500 |           1.4850 |      6.2645 |             1.1124 |      0.8926 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    7 | 06m52s | -7297.83529 |    8.8964 |             0.3185 |    0.5232 |           0.4690 |     14.3153 |            19.9606 |      0.6383 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    8 | 03m04s | -187186.92188 |    9.9944 |             0.2647 |    6.4178 |           2.8567 |      9.9968 |             6.3382 |      0.7965 | Warning: Test point chose at random due to repeated sample.

Multiple eval metrics have been passed: 'test-mae' will be used for early stop

In [10]:
# not used - reset the variable
#new_init = { 'target': [], 'alpha': [], 'colsample_bytree': [], 'gamma': [], 'max_delta_step': [], 'max_depth': [], 'min_child_weight': [], 'subsample': [] }
new_init = init_values

# store resulting values to help seed the next run. make sure not to overwrite but add incrementally
# copy paste the print out of init_values into the cell above
for i in range(len(XGB_BOpt.res['all']['values'])):
    new_init['target'].append(XGB_BOpt.res['all']['values'][i])
    for k,v in XGB_BOpt.res['all']['params'][i].items():
        new_init[k].append(v)
    
print (new_init)

{'target': [-207885.453125, -208155.27083333334, -208065.26041666666, -8482.436034999999, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -208065.26041666666, -187276.90625, -7297.8352863333339, -187186.921875, -13187.901367333334, -187186.921875, -187186.921875, -187186.921875, -187186.921875, -187186.921875, -187186.921875], 'alpha': [9.609593275137133, 0.6613706829779054, 9.99437912773472, 0.2082503541460734, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.926950421589801, 8.8964024320545558, 9.9943791277347191, 0.58608203018219007, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191], 'colsample_bytree': [0.3695897782674733, 0.8602917861499724, 0.26467482205195547, 0.2632600769188274, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.264674822

In [ ]:
%matplotlib inline
print(XGB_BOpt.res['max'])
(pd.DataFrame(XGB_BOpt.res['all']['values'])*-1.0).plot()

In [ ]:
import json
import requests
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url
from cloudinary.api import delete_resources_by_tag, resources_by_tag

def plot_rounds(plot):
    # uploads the graph to the web and returns the URL
    
    fig = plot.get_figure()
    fig.savefig('temp_plot.png')
    
    response = upload("temp_plot.png")
    url, options = cloudinary_url(response['public_id'],
        format = response['format'],
        crop = "fill")
    return url

def slack(text, url = None):
    print("Slacking: " + text)
    
    if url == None:
        data=json.dumps({"text": text})
    else:
        data = json.dumps( { "text": text, "attachments": [ { "fallback": "Model MAE"
                                           , "title": "Model Mean Average Error by Iteration ($)"
                                           , "image_url": url } ] } )
    
    response = requests.post(webhook_url, data , headers={'Content-Type': 'application/json'})
    if response.status_code != 200:
        raise ValueError('Request to slack returned an error %s, the response is:\n%s' % (response.status_code, response.text))

In [ ]:
result = pd.DataFrame(XGB_BOpt.res['all']['params'])
error = pd.Series(XGB_BOpt.res['all']['values']) * -1
error.name = 'test-mae-mean'
result = pd.concat([error, result], axis=1)
result.head(25)

url = plot_rounds(error.plot())
slack("Bayesian Search: Max params %s" % XGB_BOpt.res['max'], url)

file = 'ALL-bayesian-parameters.csv'
result.to_csv(file)
slacker.files.upload(file, channels='#progress')